<a href="https://colab.research.google.com/github/nwon24/nlp/blob/main/W4/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Using PyTorch

For some reason, the cost never goes below around 0.69. :(

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device=torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

data=pd.read_csv("corpus.csv",encoding="latin-1")

tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(data["text"])
train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data["label"],test_size=0.3)
Encoder = LabelEncoder()
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)
train_corpus_tfidf=tfidf.transform(train_corpus)
test_corpus_tfidf=tfidf.transform(test_corpus)

bsize=50
epochs=10
lr=1e-5

class dense_classifier(nn.Module):
    def __init__(self,n):
        super(dense_classifier,self).__init__()
        self.n=n
        self.network=nn.Sequential(
            nn.Linear(self.n,360),
            nn.ReLU(),
            nn.Linear(360,180),
            nn.ReLU(),
            nn.Linear(180,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.network(x)

class classifier(nn.Module):
    def __init__(self,n,kerns,pool_kerns,out_chans):
        super(classifier,self).__init__()
        self.kerns=kerns
        self.pool_kerns=pool_kerns
        self.convs=[]
        self.pools=[]
        self.stride=1
        self.n=n
        #self.embed=nn.Embedding(n,max_length)
        in_chan=1
        for i in range(len(kerns)):
            out_chan=out_chans[i]
            kern=kerns[i]
            pool_kern=pool_kerns[i]
            self.convs.append(nn.Conv1d(in_chan,out_chan,kern))
            self.compute_size(kern,self.stride)
            self.pools.append(nn.MaxPool1d(pool_kern))
            self.compute_size(pool_kern,pool_kern)
            in_chan=out_chan
        self.convs=nn.ModuleList(self.convs)
        self.pools=nn.ModuleList(self.pools)
        self.fc1=nn.Linear(self.n*out_chan,100)
        self.fc2=nn.Linear(100,1)

    def compute_size(self,kernel,stride):
        self.n=(self.n-kernel)//stride+1
        return self.n

    def forward(self,x):
        #x=self.embed(x)
        for i in range(len(self.kerns)):
            x=nn.ReLU()(self.convs[i](x))
            x=self.pools[i](x)
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        #x=nn.Sigmoid()(x)
        return x

class corpus(Dataset):
    def __init__(self,corpus,label):
        self.corpus=corpus
        self.label=label
    def __len__(self):
        return len(self.corpus)
    def __getitem__(self,idx):
        return self.corpus[idx],self.label[idx]

def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch,(x,y) in enumerate(dataloader):
        x,y=x.to(device),y.to(device)
        optimizer.zero_grad()
        #print(torch.sum(x[0]!=0))
        pred=model(x.reshape(bsize,1,x.shape[1]).to(device))
        cost=loss_fn(pred.flatten(),y)
        cost.backward()
        optimizer.step()
        if batch % 10 == 0:
            cost, current = cost.item(), batch * bsize + len(x)
            print(f"cost: {cost:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x,y=x.to(device),y.to(device)
            pred=model(x.reshape(bsize,1,x.shape[1]).to(device))
            test_loss += loss_fn(pred.flatten(), y).item()
            ncorrect = (torch.round(pred.flatten())==y).sum().item()
            print(ncorrect)
            correct += ncorrect

    #test_loss /= num_batches
    #correct /= size
    print(f"Test Error: \n Accuracy: {100*(correct/size):>0.1f}%, Avg loss: {test_loss/size:>8f} \n")

text="ajbjdfowd"
#input =torch.from_numpy(tfidf.transform([text]).toarray()).float()
train_corpus_data=torch.from_numpy(train_corpus_tfidf.toarray()).float()
test_corpus_data=torch.from_numpy(test_corpus_tfidf.toarray()).float()
train_label_data=torch.from_numpy(train_label).float()
test_label_data=torch.from_numpy(test_label).float()

train_loader=DataLoader(corpus(train_corpus_data,train_label_data),batch_size=bsize)
test_loader=DataLoader(corpus(test_corpus_data,test_label_data),batch_size=bsize)

#input=input.reshape(1,input.shape[0],input.shape[1]
#c=classifier(train_corpus_data.shape[1],[2,3,4,5],[2,2,2,2],[6,16,24,32])
cnn_classifier=classifier(train_corpus_data.shape[1],[10,20],[2,2],[5,10]).to(device)
#input=input.reshape(1,input.shape[0],input.shape[1])
#print(c.forward(input))
dense_classifier=dense_classifier(train_corpus_data.shape[1]).to(device)

c=dense_classifier

loss=nn.BCELoss()
#loss=nn.BCEWithLogitsLoss()
#optim=torch.optim.SGD(c.parameters(),momentum=0.8,lr=lr)
optim=torch.optim.Adam(c.parameters(),lr=lr)
#optim=torch.optim.ASGD(c.parameters(),lr=lr)

#print(torch.sum(train_label_data==1))

for epoch in range(epochs):
    train(train_loader,c,loss,optim)
test_loop(test_loader,c,loss)

cost: 0.694041  [   50/ 7000]
cost: 0.692548  [  550/ 7000]
cost: 0.693420  [ 1050/ 7000]
cost: 0.693384  [ 1550/ 7000]
cost: 0.693351  [ 2050/ 7000]
cost: 0.692249  [ 2550/ 7000]
cost: 0.693125  [ 3050/ 7000]
cost: 0.693475  [ 3550/ 7000]
cost: 0.693925  [ 4050/ 7000]
cost: 0.693143  [ 4550/ 7000]
cost: 0.693470  [ 5050/ 7000]
cost: 0.693638  [ 5550/ 7000]
cost: 0.693339  [ 6050/ 7000]
cost: 0.692208  [ 6550/ 7000]
cost: 0.693187  [   50/ 7000]
cost: 0.692034  [  550/ 7000]
cost: 0.692733  [ 1050/ 7000]
cost: 0.692704  [ 1550/ 7000]
cost: 0.692464  [ 2050/ 7000]
cost: 0.691785  [ 2550/ 7000]
cost: 0.692551  [ 3050/ 7000]
cost: 0.692459  [ 3550/ 7000]
cost: 0.692592  [ 4050/ 7000]
cost: 0.692199  [ 4550/ 7000]
cost: 0.691954  [ 5050/ 7000]
cost: 0.692444  [ 5550/ 7000]
cost: 0.692123  [ 6050/ 7000]
cost: 0.691025  [ 6550/ 7000]
cost: 0.691568  [   50/ 7000]
cost: 0.690668  [  550/ 7000]
cost: 0.691161  [ 1050/ 7000]
cost: 0.691105  [ 1550/ 7000]
cost: 0.690331  [ 2050/ 7000]
cost: 0.69

# Keras

Yay, it works! (A bit of overfitting is happening, but at least the first few epochs look good.)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.preprocessing import sequence
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection

data=pd.read_csv("corpus.csv",encoding="latin-1")

tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(data["text"])
train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data['label'],test_size=0.3)
Encoder = LabelEncoder()
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)
train_corpus_tfidf=tfidf.transform(train_corpus)
test_corpus_tfidf=tfidf.transform(test_corpus)

bsize=20
epochs=4
lr=1e-7

features=train_corpus_tfidf.shape[1]
train_corpus_tfidf=train_corpus_tfidf.toarray()
test_corpus_tfidf=test_corpus_tfidf.toarray()
train_corpus_tfidf=train_corpus_tfidf.reshape(train_corpus_tfidf.shape[0],train_corpus_tfidf.shape[1],1)
test_corpus_tfidf=test_corpus_tfidf.reshape(test_corpus_tfidf.shape[0],test_corpus_tfidf.shape[1],1)

model=Sequential([InputLayer(shape=(features,1),batch_size=bsize),
                  #Conv1D(64,32,activation="relu"),
                  Conv1D(6,3,activation="relu"),
                  MaxPooling1D(),
                  Dropout(0.5),
                  #Conv1D(3,3,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  #Conv1D(18,8,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  Flatten(),
                  #Dense(100,activation="relu"),
                  #Dense(50,activation="relu"),
                  Dense(1,activation="sigmoid")
                  ])
#model=Sequential([InputLayer(shape=(features,)),
#                  Dense(100,activation="relu"),
#                  Dense(1,activation="sigmoid")
#                  ])
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

model.fit(train_corpus_tfidf, train_label, epochs=epochs, batch_size=bsize, validation_data=(test_corpus_tfidf, test_label))
eval_loss,eval_accuracy=model.evaluate(test_corpus_tfidf,test_label)
print(eval_loss,eval_accuracy)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (20, 31625, 6)         │            24 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (20, 15812, 6)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (20, 15812, 6)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (20, 94872)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (20, 1)                │        94,873 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 94,897 (370.69 KB)

 Trainable params: 94,897 (370.69 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/4
350/350 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - accuracy: 0.6814 - loss: 0.6697 - val_accuracy: 0.8383 - val_loss: 0.5470
Epoch 2/4
350/350 ━━━━━━━━━━━━━━━━━━━━ 17s 43ms/step - accuracy: 0.8827 - loss: 0.4724 - val_accuracy: 0.8660 - val_loss: 0.3947
Epoch 3/4
350/350 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - accuracy: 0.9170 - loss: 0.3100 - val_accuracy: 0.8690 - val_loss: 0.3317
Epoch 4/4
350/350 ━━━━━━━━━━━━━━━━━━━━ 21s 43ms/step - accuracy: 0.9405 - loss: 0.2226 - val_accuracy: 0.8640 - val_loss: 0.3110
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8593 - loss: 0.3091
0.3109709620475769 0.8640000224113464
